In [10]:
import pandas as pd
import numpy as np
import math



In [11]:
# Paso 2: Cargar los datos desde el archivo CSV
file_path = 'VolTrend-strategy-V2_List_of_Trades_2024-09-24.csv'  # Cambia por la ubicación de tu archivo
df = pd.read_csv(file_path)
df.head()


,Trade #,Type,Signal,Date/Time,Price USDT,Contracts,Profit USDT,Profit %,Cum. Profit USDT,Cum. Profit %,Run-up USDT,Run-up %,Drawdown USDT,Drawdown %
0,792,Exit Long,Close entry(s) order Long,2024-09-23 06:00,63492.00,0.03344,10.97,0.52,5560.29,0.01,54.60,2.59,25.25,1.20
1,792,Entry Long,Long,2024-09-22 15:00,63062.66,0.03344,10.97,0.52,5560.29,0.01,54.60,2.59,25.25,1.20
2,791,Exit Long,Close entry(s) order Long,2024-09-22 00:00,62957.31,0.03340,-9.47,-0.45,5549.32,-0.01,12.34,0.58,12.04,0.57
3,791,Entry Long,Long,2024-09-21 18:00,63139.99,0.03340,-9.47,-0.45,5549.32,-0.01,12.34,0.58,12.04,0.57
4,790,Exit Long,Close entry(s) order Long,2024-09-21 18:00,63139.99,0.03332,-8.70,-0.41,5558.79,-0.01,1.64,0.08,9.62,0.46


In [12]:
# Paso 3: Filtrar las columnas necesarias para los cálculos
# Aquí usamos solo las columnas relevantes: 'Profit USDT', 'Profit %', 'Drawdown USD', 'Drawdown %'
df_clean = df[['Profit USDT', 'Profit %', 'Drawdown USDT', 'Drawdown %']].dropna()

# Convertimos las columnas a valores numéricos para evitar errores en los cálculos posteriores
df_clean['Profit USD'] = pd.to_numeric(df_clean['Profit USDT'], errors='coerce')
df_clean['Profit %'] = pd.to_numeric(df_clean['Profit %'], errors='coerce')
df_clean['Drawdown USD'] = pd.to_numeric(df_clean['Drawdown USDT'], errors='coerce')
df_clean['Drawdown %'] = pd.to_numeric(df_clean['Drawdown %'], errors='coerce')

# Filtramos nuevamente por posibles valores nulos o inconsistentes
df_clean = df_clean.dropna()
df_clean


,Profit USDT,Profit %,Drawdown USDT,Drawdown %,Profit USD,Drawdown USD
0,10.97,0.52,25.25,1.20,10.97,25.25
1,10.97,0.52,25.25,1.20,10.97,25.25
2,-9.47,-0.45,12.04,0.57,-9.47,12.04
3,-9.47,-0.45,12.04,0.57,-9.47,12.04
4,-8.70,-0.41,9.62,0.46,-8.70,9.62
...,...,...,...,...,...,...
1579,-3.20,-0.16,1.60,0.08,-3.20,1.60
1580,35.43,1.77,5.08,0.25,35.43,5.08
1581,35.43,1.77,5.08,0.25,35.43,5.08
1582,-3.20,-0.16,1.60,0.08,-3.20,1.60


In [13]:
 # Paso 3: Filtrar las filas y columnas necesarias para los cálculos

# Eliminamos las filas donde 'Type' contiene 'Entry'
df_clean = df[~df['Type'].str.contains('Entry', case=False)].copy()

# Mantener solo las columnas relevantes: 'Type', 'Profit USDT', 'Profit %', 'Drawdown USD', 'Drawdown %'
df_clean = df_clean[['Type', 'Profit USDT', 'Profit %', 'Drawdown USDT', 'Drawdown %']]

# Convertimos las columnas de ganancias y drawdowns a valores numéricos
df_clean['Profit USD'] = pd.to_numeric(df_clean['Profit USDT'], errors='coerce')
df_clean['Profit %'] = pd.to_numeric(df_clean['Profit %'], errors='coerce')
df_clean['Drawdown USD'] = pd.to_numeric(df_clean['Drawdown USDT'], errors='coerce')
df_clean['Drawdown %'] = pd.to_numeric(df_clean['Drawdown %'], errors='coerce')

# Filtramos nuevamente para eliminar cualquier fila con valores nulos o no numéricos
df_clean = df_clean.dropna()

# Mostrar las primeras filas del dataframe limpio para verificar
df_clean.head()


,Type,Profit USDT,Profit %,Drawdown USDT,Drawdown %,Profit USD,Drawdown USD
0,Exit Long,10.97,0.52,25.25,1.20,10.97,25.25
2,Exit Long,-9.47,-0.45,12.04,0.57,-9.47,12.04
4,Exit Long,-8.70,-0.41,9.62,0.46,-8.70,9.62
6,Exit Long,2.50,0.12,8.69,0.41,2.50,8.69
8,Exit Long,62.60,2.97,5.11,0.24,62.60,5.11


In [14]:
# Paso 4: Cálculo del Sharpe Ratio
# Fórmula del Sharpe Ratio: media de los retornos dividida por la desviación estándar de los retornos
# Esto mide el rendimiento ajustado por riesgo (volatilidad)

mean_profit = df_clean['Profit %'].mean()  # Calcula la media de los retornos (en porcentaje)
std_profit = df_clean['Profit %'].std()    # Calcula la desviación estándar de los retornos
sharpe_ratio = mean_profit / std_profit    # Sharpe Ratio es media sobre desviación estándar


In [15]:
# Paso 5: Cálculo del Sortino Ratio
# Similar al Sharpe Ratio, pero usando solo la desviación estándar de los retornos negativos (downside)

downside_std = df_clean[df_clean['Profit %'] < 0]['Profit %'].std()  # Desviación de pérdidas
sortino_ratio = mean_profit / downside_std if downside_std != 0 else np.nan  # Evitar división por cero


In [16]:
# Paso 6: Cálculo del Calmar Ratio
# Calmar Ratio = retorno anualizado dividido por el máximo drawdown (pérdida máxima)
# Supongo que hay 252 días de trading en un año para anualizar los retornos

annual_return = mean_profit * 252  # Retorno anualizado
max_drawdown = df_clean['Drawdown %'].max()  # Máximo drawdown
calmar_ratio = annual_return / max_drawdown  # Calmar Ratio es el retorno sobre el drawdown máximo


In [17]:
# Paso 7: Cálculo del SQN (System Quality Number)
# SQN = (media de los retornos / desviación estándar) * raíz cuadrada del número de trades

n_trades = len(df_clean)  # Número de transacciones
sqn = (mean_profit / std_profit) * np.sqrt(n_trades)  # Fórmula completa del SQN


In [18]:
# Paso 8: Mostrar los resultados de las métricas calculadas
# Generamos un diccionario con los resultados para facilitar la visualización

resultados = {
    "Sharpe Ratio": sharpe_ratio,
    "Sortino Ratio": sortino_ratio,
    "Calmar Ratio": calmar_ratio,
    "SQN": sqn
}

# Mostramos los resultados
resultados


{'Sharpe Ratio': np.float64(0.15540981173894866),
 'Sortino Ratio': np.float64(0.36754072947763095),
 'Calmar Ratio': np.float64(10.551018137309446),
 'SQN': np.float64(4.373619781269342)}